In [1]:
class InformationManagementExpert:
    def __init__(self):
        self.data = {}

    def add_information(self, key, value):
        self.data[key] = value

    def get_information(self, key):
        if key in self.data:
            return self.data[key]
        else:
            return None

    def remove_information(self, key):
        if key in self.data:
            del self.data[key]

    def get_all_information(self):
        return self.data


# Usage example:
info_expert = InformationManagementExpert()

# Adding information
info_expert.add_information("Name", "John Doe")
info_expert.add_information("Age", 25)
info_expert.add_information("City", "New York")

# Getting individual information
name = info_expert.get_information("Name")
age = info_expert.get_information("Age")
city = info_expert.get_information("City")
print("Name:", name)
print("Age:", age)
print("City:", city)

# Removing information
info_expert.remove_information("Age")

# Getting all information
all_info = info_expert.get_all_information()
print("All Information:")
for key, value in all_info.items():
    print(key + ":", value)


Name: John Doe
Age: 25
City: New York
All Information:
Name: John Doe
City: New York
